In [ ]:
# Importamos PyTorch y submódulos
import torch                     # Núcleo de PyTorch: tensores, operaciones y autograd
import torch.nn as nn            # Para construir redes neuronales (capas, activaciones)
import torch.optim as optim      # Optimización (SGD, Adam, etc.)
from torchvision import datasets, transforms  # Para cargar datasets y transformaciones

# Transformación: convierte imágenes a tensores y normaliza valores a [0,1]
transform = transforms.ToTensor()

# Cargamos y obtenemos el dataset MNIST (imágenes 28x28 de dígitos 0-9)
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Creamos "DataLoaders" para manejar lotes de datos y mezclar el dataset
# nos trae los datos en pequeños batch 64 imag para entrenar, shuffle mezcla imagenes
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=False)

100.0%
100.0%
100.0%
100.0%


In [ ]:
# Definimos nuestra red neuronal (MLP) hereda de nn.module clase base
class MLP(nn.Module):
    def __init__(self):
        # llama al contructor de la clase padre nn.module
        super().__init__()
        # Secuencia de capas: Flatten → Linear → ReLU → Linear → ReLU → Linear
        self.layers = nn.Sequential(
            nn.Flatten(),          # Convierte imagen 28x28 a vector de 784 elementos
            nn.Linear(28*28, 128), # Capa oculta de 128 neuronas
            nn.ReLU(),             # Función de activación ReLU
            nn.Linear(128, 64),    # Segunda capa oculta de 64 neuronas
            nn.ReLU(),             # Activación ReLU
            nn.Linear(64, 10)      # Capa de salida: 10 clases (dígitos 0-9)
        )
    # x es un batch de imagenes, self.layers aplica a las capas un oorden.
    def forward(self, x):
        # Devuelve un tensor de forma batch 10 prediccion de probabilidad para cada clase 0-9
        return self.layers(x) # Define el flujo de datos hacia adelante 
    
# Creamos una instancia del modelo
model = MLP()

In [ ]:
# Definimos la función de pérdida (cross-entropy para clasificación multi-clase)
# Mide que tan mal predice el modelo.
criterion = nn.CrossEntropyLoss()

# Definimos el optimizador (Adam) para actualizar los pesos del modelo
# Es el motor del aprendizaje: mira los errores, ajusta los pesos de la red, en resumen es quien aprende.
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# --------- Entrenamiento ---------
for epoch in range(5):  # Iteramos 5 veces sobre todo el dataset
    model.train()        # Indicamos que estamos entrenando (activa dropout, batchnorm si hubiera)
    total_loss = 0       # Para acumular la pérdida de cada batch
    for X, y in train_loader:  # Iteramos por cada lote de imágenes y etiquetas
        optimizer.zero_grad()   # Reiniciamos los gradientes(errores) del paso anterior
        y_pred = model(X)      # Pasamos el batch por la red (forward pass) = hacer prediccion
        loss = criterion(y_pred, y)  # Calculamos la pérdida entre predicción y etiqueta
        loss.backward()        # Calculamos gradientes(errores) (backpropagation)
        optimizer.step()       # Actualizamos pesos según el optimizador
        total_loss += loss.item()  # Acumulamos la pérdida del batch, perdida promedio
    # Mostramos la pérdida promedio de la época
    # Por cada iteracion deberia mostrar una prdida mas baja
    print(f"Época {epoch+1}, pérdida promedio: {total_loss/len(train_loader):.4f}")

Época 1, pérdida promedio: 0.3414
Época 2, pérdida promedio: 0.1423
Época 3, pérdida promedio: 0.0972
Época 4, pérdida promedio: 0.0728
Época 5, pérdida promedio: 0.0570


In [ ]:
# --------- Evaluación ---------
correct = 0
total = 0
model.eval()   # Indicamos que estamos evaluando (desactiva dropout, batchnorm)
with torch.no_grad():  # No necesitamos calcular gradientes(errores) en evaluación
    for X, y in test_loader:
        y_pred = model(X)             # Forward pass
        _, predicted = torch.max(y_pred, 1)  # Tomamos la clase con mayor probabilidad
        total += y.size(0)            # Contamos número total de imágenes
        correct += (predicted == y).sum().item()  # Contamos aciertos

# Mostramos precisión en el dataset de test
print(f"Precisión en test: {100 * correct / total:.2f}%")

Precisión en test: 97.25%
